# 주제: 유형별 임대주택 설계 시 단지 내 적정 주차 수요를 예측

- 아파트 단지 내 필요한 주차대수는 </br>
a).법정주차대수 b).장래주차수요 중 큰 값에 따라 결정하게되어 있어 정확한 </br>
b).장래주차수요의 산정을 필요로 함

- 현재 b).장래주차수요는 ‘주차원단위’와 ‘건축연면적’을 기초로하여 산출되고 있으며, ‘주차원단위’는 신규 건축예정 부지 인근의 유사 단지를 피크 시간대 방문하여 주차된 차량대수를 세는 방법으로 조사하고 있음

- 이 경우 인력조사로 인한 오차발생, 현장조사 시점과 실제 건축시점과의 시간차 등의 문제로 과대 또는 과소 산정의 가능성을 배제할 수 없음

- 본 분석에서는 상가건축물은 대상에서 제외하였음

## 1.데이터 탐색

### 1.1 데이터 특성 파악

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# train 및 test 설정
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# 길이, 특성, 결측치 등 전체적인 구조 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

- train 데이터에서는 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수에 NULL값 존재

In [ ]:
# train 상위 다섯 개 확인
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
test.info() # 이 부분 실시해서 결측값 있는지 확인해주세요

- 만약 결측치가 있다면 훈련 데이터와 같이 어떤 컬럼이 결측값을 갖고 있는지 써주시면 좋을거 같아요.

In [ ]:
# null이 존재하는 컬럼 확인 (결측치가 없다면 false, 있다면 true)
train.isnull().any(axis=0)

단지코드                            False
총세대수                            False
임대건물구분                          False
지역                              False
공급유형                            False
전용면적                            False
전용면적별세대수                        False
공가수                             False
자격유형                            False
임대보증금                            True
임대료                              True
도보 10분거리 내 지하철역 수(환승노선 수 반영)     True
도보 10분거리 내 버스정류장 수               True
단지내주차면수                         False
등록차량수                           False
dtype: bool

In [ ]:
# 컬럼별로 null개수 확인하기 (결측값 존재시 어떻게 처리할지 생각하기)
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [ ]:
train.dtypes
# 임대보증금과 임대료 타입 object임을 확인

단지코드                             object
총세대수                              int64
임대건물구분                           object
지역                               object
공급유형                             object
전용면적                            float64
전용면적별세대수                          int64
공가수                             float64
자격유형                             object
임대보증금                            object
임대료                              object
도보 10분거리 내 지하철역 수(환승노선 수 반영)    float64
도보 10분거리 내 버스정류장 수              float64
단지내주차면수                         float64
등록차량수                           float64
dtype: object

In [ ]:
# 임대보증금과 임대료 타입 float로 변환

train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-','임대보증금' ] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)
train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

In [ ]:
# 변환된 것 확인
train.dtypes

단지코드                             object
총세대수                              int64
임대건물구분                           object
지역                               object
공급유형                             object
전용면적                            float64
전용면적별세대수                          int64
공가수                             float64
자격유형                             object
임대보증금                           float64
임대료                             float64
도보 10분거리 내 지하철역 수(환승노선 수 반영)    float64
도보 10분거리 내 버스정류장 수              float64
단지내주차면수                         float64
등록차량수                           float64
dtype: object

In [ ]:
test.dtypes

단지코드                             object
총세대수                              int64
임대건물구분                           object
지역                               object
공급유형                             object
전용면적                            float64
전용면적별세대수                          int64
공가수                             float64
자격유형                             object
임대보증금                           float64
임대료                             float64
도보 10분거리 내 지하철역 수(환승노선 수 반영)    float64
도보 10분거리 내 버스정류장 수              float64
단지내주차면수                         float64
dtype: object

### 1.2 결측치 관련

In [ ]:
train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0, inplace=True)
train['도보 10분거리 내 버스정류장 수'].fillna(0, inplace=True)

In [ ]:
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0, inplace=True)
test['도보 10분거리 내 버스정류장 수'].fillna(0, inplace=True)

- 지하철역과 버스 정류장의 Null 값의 경우는, 주변에 없기 때문에 0으로 했을 것이라 판단 ➡ fillna를 통해 0으로 채움

In [ ]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           581
임대료                             590
도보 10분거리 내 지하철역 수(환승노선 수 반영)      0
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [ ]:
test.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           186
임대료                             186
도보 10분거리 내 지하철역 수(환승노선 수 반영)      0
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

In [ ]:
# test의 자격유형에 있는 2개의 결측치 채우기
test.loc[(test.단지코드=='C2411')&(test.자격유형.isnull()), '자격유형'] = 'A'
test.loc[(test.단지코드=='C2253')&(test.자격유형.isnull()), '자격유형'] = 'C'

In [ ]:
test.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           186
임대료                             186
도보 10분거리 내 지하철역 수(환승노선 수 반영)      0
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

## 2.본격적인 훈련세트, 테스트세트 생성

### 2.1 임대건물구분이 '아파트'인 데이터들로 구성된 train 세트 재정의

In [ ]:
train = train[train.임대건물구분 == '아파트']
test = test[test.임대건물구분 == '아파트']

In [ ]:
train.isnull().sum()

단지코드                             0
총세대수                             0
임대건물구분                           0
지역                               0
공급유형                             0
전용면적                             0
전용면적별세대수                         0
공가수                              0
자격유형                             0
임대보증금                           19
임대료                             28
도보 10분거리 내 지하철역 수(환승노선 수 반영)     0
도보 10분거리 내 버스정류장 수               0
단지내주차면수                          0
등록차량수                            0
dtype: int64

In [ ]:
test.isnull().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           9
임대료                             9
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
dtype: int64

In [ ]:
train.loc[(train.공급유형=='장기전세')&(train.임대료.isnull()), '임대료']=0

In [ ]:
train.isnull().sum()
# 임대보증금 임대료의 개수가 동일해졌음을 확인

단지코드                             0
총세대수                             0
임대건물구분                           0
지역                               0
공급유형                             0
전용면적                             0
전용면적별세대수                         0
공가수                              0
자격유형                             0
임대보증금                           19
임대료                             19
도보 10분거리 내 지하철역 수(환승노선 수 반영)     0
도보 10분거리 내 버스정류장 수               0
단지내주차면수                          0
등록차량수                            0
dtype: int64

In [ ]:
print(len(train[train.공급유형 == '공공분양']))

7


In [ ]:
# 임대료가 NaN인 값을 모은 데이터프레임
null_train= train[train['임대료'].isnull()]

In [ ]:
null_train
# 임대보증금, 임대료가 NaN인 데이터들 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0
2548,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0
2549,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.0,H,NaN,NaN,0.0,4.0,1670.0,1153.0


In [ ]:
null_train.shape

(19, 15)

In [ ]:
train = train[(train.임대건물구분 == '아파트') & (train.공급유형 != '공공분양')]

In [ ]:
train.isnull().sum()

단지코드                             0
총세대수                             0
임대건물구분                           0
지역                               0
공급유형                             0
전용면적                             0
전용면적별세대수                         0
공가수                              0
자격유형                             0
임대보증금                           12
임대료                             12
도보 10분거리 내 지하철역 수(환승노선 수 반영)     0
도보 10분거리 내 버스정류장 수               0
단지내주차면수                          0
등록차량수                            0
dtype: int64

In [ ]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0


In [ ]:
print(np.unique(train['공급유형'].to_numpy(), return_counts=True))

(array(['공공임대(10년)', '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대',
       '장기전세', '행복주택'], dtype=object), array([ 205,   31,    3,   12, 1758,  152,    9,  213]))


In [ ]:
# 혜원이가 정리한 내용 표로 보기
train.groupby('공급유형')['공가수'].agg(**{'count_':'count'}).reset_index()

,공급유형,count_
0,공공임대(10년),205
1,공공임대(50년),31
2,공공임대(5년),3
3,공공임대(분납),12
4,국민임대,1758
5,영구임대,152
6,장기전세,9
7,행복주택,213


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2383 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2383 non-null   object 
 1   총세대수                          2383 non-null   int64  
 2   임대건물구분                        2383 non-null   object 
 3   지역                            2383 non-null   object 
 4   공급유형                          2383 non-null   object 
 5   전용면적                          2383 non-null   float64
 6   전용면적별세대수                      2383 non-null   int64  
 7   공가수                           2383 non-null   float64
 8   자격유형                          2383 non-null   object 
 9   임대보증금                         2371 non-null   float64
 10  임대료                           2371 non-null   float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2383 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2383 non-null   float64
 13  단지내

In [ ]:
train['임대건물구분'].value_counts()

아파트    2383
Name: 임대건물구분, dtype: int64

In [ ]:
train.dtypes

단지코드                             object
총세대수                              int64
임대건물구분                           object
지역                               object
공급유형                             object
전용면적                            float64
전용면적별세대수                          int64
공가수                             float64
자격유형                             object
임대보증금                           float64
임대료                             float64
도보 10분거리 내 지하철역 수(환승노선 수 반영)    float64
도보 10분거리 내 버스정류장 수              float64
단지내주차면수                         float64
등록차량수                           float64
dtype: object

In [ ]:
test.isnull().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           9
임대료                             9
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
dtype: int64

In [ ]:
train.isnull().sum()

단지코드                             0
총세대수                             0
임대건물구분                           0
지역                               0
공급유형                             0
전용면적                             0
전용면적별세대수                         0
공가수                              0
자격유형                             0
임대보증금                           12
임대료                             12
도보 10분거리 내 지하철역 수(환승노선 수 반영)     0
도보 10분거리 내 버스정류장 수               0
단지내주차면수                          0
등록차량수                            0
dtype: int64

In [ ]:
# 중복 개수 확인
train.shape, train.drop_duplicates().shape

((2383, 15), (2340, 15))

In [ ]:
test.shape, test.drop_duplicates().shape

((845, 14), (837, 14))

In [ ]:
# 중복 값 제거
train = train.drop_duplicates()
test = test.drop_duplicates()

In [ ]:
# 중복 값 제거 확인
print(train.shape, train.drop_duplicates().shape)
print(test.shape, test.drop_duplicates().shape)

(2340, 15) (2340, 15)
(837, 14) (837, 14)


## 3.주택 수정
- 지역별 전용면적을 기준으로 임대보증금, 임대료 조정
- https://dacon.io/competitions/official/235745/codeshare/3017?page=1&dtype=recent 참고

### 3.1train 세트 수정

In [ ]:
# 강원도 
# 전용면적 16.91 
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 16.91), '임대보증금'] = 13450000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 16.91), '임대료'] = 65500
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 19700000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 96000
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 19150000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 94000
# 전용면적 26.9
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대보증금'] = 21400000
train.loc[(train['단지코드'] == 'C1786') & (train['전용면적'] == 26.9), '임대료'] = 105000

In [ ]:
# 부산
# 전용면적 24.72
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.72), '임대보증금'] = 7000000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.72), '임대료'] = 135000
# 전용면적 24.79
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.79), '임대보증금'] = 7000000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 24.79), '임대료'] = 135000
# 전용면적 26.83
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 26.83), '임대보증금'] = 7600000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 26.83), '임대료'] = 142000
# 전용면적 37.7
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 37.7), '임대보증금'] = 14800000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 37.7), '임대료'] = 198000
# 전용면적 46.94
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 46.94), '임대보증금'] = 23100000
train.loc[(train['단지코드'] == 'C1326') & (train['전용면적'] == 46.94), '임대료'] = 259000

In [ ]:
# 대구
# 전용면적 29.17
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.17), '임대보증금'] = 10847000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.17), '임대료'] = 138600
# 전용면적 29.34
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.34), '임대보증금'] = 10847000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 29.34), '임대료'] = 138600
# 전용면적 37.43
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 37.43), '임대보증금'] = 17338000
train.loc[(train['단지코드'] == 'C2186') & (train['전용면적'] == 37.43), '임대료'] = 197500

In [ ]:
# 임대료가 NaN인 값을 모은 데이터프레임
null_train= train[train['임대료'].isnull()]
null_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수


In [ ]:
train.isnull().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           0
임대료                             0
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
등록차량수                           0
dtype: int64

### 3.2test 수정

In [ ]:
# 강원도
# 전용면적 24.83
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 24.83), '임대보증금'] = 2129000
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 24.83), '임대료'] = 42350
# 전용면적 33.84
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 33.84), '임대보증금'] = 2902000
test.loc[(test['단지코드'] == 'C2152') & (test['전용면적'] == 33.84), '임대료'] = 57730

In [ ]:
# 경상남도
# 전용면적 16.94
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 16.94), '임대보증금'] = 11200000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 16.94), '임대료'] = 53200
# 전용면적 26.85
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대보증금'] = 16333330
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대료'] = 77580
# 전용면적 26.85
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대보증금'] = 18620000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 26.85), '임대료'] = 88440
# 전용면적 36.77
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 36.77), '임대보증금'] = 23760000
test.loc[(test['단지코드'] == 'C1267') & (test['전용면적'] == 36.77), '임대료'] = 112860

In [ ]:
# 대전

# 자격유형 C로 변경

# 전용면적 26.37
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '임대보증금'] = 5787000
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '임대료'] = 79980
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 26.37), '자격유형'] = 'C'
# 전용면적 52.74
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '임대보증금'] = 11574000
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '임대료'] = 159960
test.loc[(test['단지코드'] == 'C1006') & (test['전용면적'] == 52.74), '자격유형'] = 'C'

In [ ]:
test.isnull().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           0
임대료                             0
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
dtype: int64

In [ ]:
train[train['단지코드']=='C2085']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2186,C2085,969,아파트,서울특별시,공공임대(10년),51.96,63,0.0,A,47001000.0,602270.0,0.0,3.0,1590.0,1595.0
2187,C2085,969,아파트,서울특별시,공공임대(10년),51.99,41,0.0,A,47001000.0,602270.0,0.0,3.0,1590.0,1595.0
2188,C2085,969,아파트,서울특별시,공공임대(10년),59.93,315,0.0,A,61210000.0,695170.0,0.0,3.0,1590.0,1595.0
2189,C2085,969,아파트,서울특별시,공공임대(분납),74.80,208,0.0,A,120611000.0,293490.0,0.0,3.0,1590.0,1595.0
2190,C2085,969,아파트,서울특별시,공공임대(분납),84.05,20,0.0,A,135521000.0,329770.0,0.0,3.0,1590.0,1595.0
2191,C2085,969,아파트,서울특별시,공공임대(분납),84.05,4,0.0,A,135531000.0,329790.0,0.0,3.0,1590.0,1595.0
2192,C2085,969,아파트,서울특별시,공공임대(분납),84.39,314,0.0,A,136646000.0,332510.0,0.0,3.0,1590.0,1595.0
2193,C2085,969,아파트,서울특별시,공공임대(분납),84.39,4,0.0,A,136628000.0,332460.0,0.0,3.0,1590.0,1595.0


### 3.3 오류데이터 삭제 (단지코드)

- 3가지 오류 유형에 대한 문제 식별
  - 첫째 전용면적별 세대수 합계와 총세대수가 일치하지 않는 경우

  - 둘째 동일한 단지에 단지코드가 2개로 부여되어 단지코드, 총세대수, 주차면수, 등록차량대수 등에서 오류 발생

  - 셋째 단지코드 등 기입 실수로 단지코드, 총세대수, 주차면수, 등록차량대수 등에서 오류 발생

In [ ]:
# 오류 데이터 삭제
error_data = ['C2085', 'C1397', 'C2431', 'C1649', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
for error in error_data:
    error_index = train[train['단지코드'] == error].index
    train.drop(error_index, inplace=True)

In [ ]:
train.shape, test.shape

((2264, 15), (837, 14))